In [ ]:
# Supplementary Tables

This notebook is to extract specific numbers about how well WALDO performs in relation to MWT

performance:
    
MWT: how many tracks in total?
MWT: how many track move > 1 bl?
WALDO: how many tracks in total?
WALDO: how many track move > 1 bl?

    
For waldo, how many of these operations are used?
consolidation/pruning/collision/infer arcs

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import print_function, division
import six, sys
sys.path.append('..'); import pathcustomize, about
about.about()

import waldo

Python 2.7.9 (default, Apr 14 2015 12:54:25) [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)] on linux2, Host: cody


In [3]:
import pathlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from waldo.conf import settings
from waldo.wio.experiment import Experiment

## Grab data from filesystem

In [4]:
#settings.PROJECT_DATA_ROOT = '/Users/peterwinter/Data/waldo_out'
#settings.PROJECT_DATA_ROOT = '../../data/test/'
settings.PROJECT_DATA_ROOT = '/home/projects/worm_movement/Data/WALDO_Data/'


pl = pathlib.Path(settings.PROJECT_DATA_ROOT)
pl.absolute()

def grab_report_card(eid):
    path = pl / eid
    report_path = path / 'waldo' / '{eid}-report-card.csv'.format(eid=eid)
    report_path.exists()

    if not report_path.exists():
        report_card = None
        print('Warning: Report Card file not found for {eid}'.format(eid=eid))
        print(report_path)
    else:
        #print('report file found')
        report_card = pd.read_csv(str(report_path))
    return report_card

def data_from_summary_file(summary_file):
    recording_name = summary_file.name

    with open(str(summary_file)) as f:
        lines = f.readlines()
    last_line = lines[-1].split()
    last_time = float(last_line[1]) / 60.0 # file is in seconds, we use minutes
    
    data = {'name': recording_name.split('.summary')[0],
            #'summary_path': str(summary_file),
            '#worms':0,
            'total-minutes':round(last_time, ndigits=1)}    
    
    name_parts = recording_name.split('-')
    for part in name_parts:
        if 'worm' in part:
            p = part.split('worm')
            for i in p:
                #print(i)
                try:
                    data['#worms'] = int(i)
                    break
                except:
                    pass

    return data

not_a_dir = []
complete_runs = []
incomplete_runs = []
eid_data = {}
report_cards = {}
for i in pl.glob('*'):
    
    # if not a directory, dont bother
    if not i.is_dir():
        not_a_dir.append(str(i))
        continue
        
    # if no summary file, skip
    summary_list = list((i / 'blob_files').glob('*.summary'))
    if not summary_list:
        incomplete_runs.append(eid)
        continue
        
    # if no report_card, skip
    eid = i.parts[-1]
    report_card = grab_report_card(eid)
    if report_card is None:
        incomplete_runs.append(eid)
        continue
    
    datetime = eid.split('_')
    if len(datetime) != 2:
        continue
    date, time = datetime
    if date[:5] != '20150':
        continue

    summary_file = summary_list[0]
    complete_runs.append(eid)            
    eid_data[eid] = data_from_summary_file(summary_file)
    report_cards[eid] = report_card
        
organization_table = pd.DataFrame(eid_data).T
organization_table.sort
organization_table

,#worms,name,total-minutes
20150512_110526,10,copper-10worms-N2-day2-standard,180
20150514_121513,10,copper-10worms-N2-day4-standard_C,180
20150514_164204,10,copper-10worms-N2-day4-standard_C,180
20150515_124728,10,copper-10worms-N2-day5-standard_B,180
20150515_124736,10,copper-10worms-N2-day5-standard_C,180
20150515_124840,10,copper-10worms-N2-day5-standard_A_Correct,180
20150515_155539,10,copper-10worms-N2-day5-standard_A,180
20150515_155545,10,copper-10worms-N2-day5-standard_C,180
20150518_121756,10,copper-10worms-N2-day1-standard_15degree,180
20150518_121757,10,copper-10worms-N2-day1-standard_15degree,180


## Manual Corrections

In [5]:
#organization_table.loc['20150504_123807', '#worms'] = 5

## Grab Important Information

In [6]:
def report_card_to_operation_metrics(report_card):
    rc = report_card[['phase', 'step', 'total-nodes']]
    rc['node_change'] = rc['total-nodes'].diff()
    operations = ['resolve collisions', 'prune', 'consolidate', 'infer gaps']
    operation_counts = {}

    for op in operations:
        r = rc[rc['step'] == op]
        count = np.abs(r['node_change'].sum())
        op_name = op.replace(' ', '-')
        operation_counts[op_name] = count
    #operation_counts['resolve-collisions'] = operation_counts['resolve-collisions'] / 3
    operation_counts['total-removed'] = np.sum(operation_counts.values())
    return operation_counts



def report_card_to_performance_metrics(report_card, worm_count = 10):

    # grab relevant rows
    MWT_row = report_card.iloc[0]
    WALDO_row = report_card.iloc[-1]

    # start calculating stats 
    metrics = {'MWT_total_tracks': MWT_row['total-nodes'],
               'MWT_moving_tracks': MWT_row['moving-nodes'],
               'WALDO_total_tracks': WALDO_row['total-nodes'],
               'WALDO_moving_tracks': WALDO_row['moving-nodes'],
               }
    
    metrics['worm_count'] = worm_count
    metrics['#interuptions'] = metrics['MWT_total_tracks'] - worm_count
    metrics['#interuptions_removed'] = metrics['MWT_total_tracks'] - metrics['WALDO_total_tracks']
    metrics['%interuptions_removed'] = int(round(100 * metrics['#interuptions_removed'] / metrics['#interuptions']))
    return metrics


report_card = report_cards.values()[0]
performance_metrics = report_card_to_performance_metrics(report_card)
print(performance_metrics)

operation_metrics = report_card_to_operation_metrics(report_card)
operation_metrics


{'WALDO_total_tracks': 22, '%interuptions_removed': 99, '#interuptions_removed': 1094, 'MWT_moving_tracks': 97, '#interuptions': 1106, 'MWT_total_tracks': 1116, 'worm_count': 10, 'WALDO_moving_tracks': 17}


/home/visitors/peterwinter/.pyenv/versions/anaconda-2.1.0/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


{'consolidate': 722.0,
 'infer-gaps': 2.0,
 'prune': 1.0,
 'resolve-collisions': 369.0,
 'total-removed': 1094.0}

## Fold increase

In [32]:
# Fold increase in tracks > 50 min
a = []
for eid, report_card in report_cards.items():
    #print(report_card.head())
    l = {'before': report_card.loc[2]['>50min'],
         'after': report_card.loc[len(report_card) - 1]['>50min'],
         'eid':eid}
    a.append(l)
df = pd.DataFrame(a).set_index('eid')[['before', 'after']]
df['fold'] = df['after'] / df['before']

before = sum(df['before'])
after = sum(df['after'])
print('before', before)
print('after', after)
print(after/ before)

df.head()

before 159
after 471
2.96226415094


,before,after,fold
eid,,,
20150615_170754,8,10,1.250000
20150608_165523,9,60,6.666667
20150602_112810,0,0,NaN
20150518_121809,11,12,1.090909
20150615_170759,2,12,6.000000


In [7]:
performance_data = []
operation_data = []

for eid, row in organization_table.iterrows():
    row_dict = row.to_dict()
    row_dict['id'] = eid
    
    report_card = report_cards[eid]
    if report_card is None:
        print('WARNING:', eid, 'has no report card')
        continue
    
    performance_metrics = report_card_to_performance_metrics(report_card, worm_count=row['#worms'])
    operation_metrics = report_card_to_operation_metrics(report_card)
    operation_metrics['MWT_total_tracks'] = performance_metrics['MWT_total_tracks']
    operation_metrics['WALDO_total_tracks'] = performance_metrics['WALDO_total_tracks']

    performance_metrics.update(row_dict)
    operation_metrics.update(row_dict)
    performance_data.append(performance_metrics)
    operation_data.append(operation_metrics)
    
    
op_unformatted = pd.DataFrame(operation_data)
pf_unformatted = pd.DataFrame(performance_data)

/home/visitors/peterwinter/.pyenv/versions/anaconda-2.1.0/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


## Format Tables

In [8]:
op = op_unformatted.set_index('id')
op = op[['name', 'total-minutes', '#worms', 'MWT_total_tracks',
         'prune', 'consolidate', 'resolve-collisions', 'infer-gaps', 
         'total-removed', 'WALDO_total_tracks']]
op.to_html('table_of_network_operations.html')
op

,name,total-minutes,#worms,MWT_total_tracks,prune,consolidate,resolve-collisions,infer-gaps,total-removed,WALDO_total_tracks
id,,,,,,,,,,
20150512_110526,copper-10worms-N2-day2-standard,180.0,10,860,4,240,495,8,747,47
20150514_121513,copper-10worms-N2-day4-standard_C,180.0,10,1041,0,339,642,1,982,55
20150514_164204,copper-10worms-N2-day4-standard_C,180.0,10,1454,1,192,609,8,810,41
20150515_124728,copper-10worms-N2-day5-standard_B,180.0,10,1083,19,361,534,7,921,23
20150515_124736,copper-10worms-N2-day5-standard_C,180.0,10,1571,27,467,837,12,1343,79
20150515_124840,copper-10worms-N2-day5-standard_A_Correct,180.0,10,1181,5,322,780,2,1109,32
20150515_155539,copper-10worms-N2-day5-standard_A,180.0,10,654,0,295,330,3,628,18
20150515_155545,copper-10worms-N2-day5-standard_C,180.0,10,618,1,173,372,3,549,20
20150518_121756,copper-10worms-N2-day1-standard_15degree,180.0,10,1014,4,472,312,29,817,52


In [9]:
pf = pf_unformatted.set_index('id')
pf = pf[['name', 'total-minutes', '#worms', 
         'MWT_total_tracks', 'WALDO_total_tracks',
         'MWT_moving_tracks', 'WALDO_moving_tracks',
         '#interuptions', '#interuptions_removed', '%interuptions_removed']]
pf.to_html('table_of_waldo_performance.html')
pf

,name,total-minutes,#worms,MWT_total_tracks,WALDO_total_tracks,MWT_moving_tracks,WALDO_moving_tracks,#interuptions,#interuptions_removed,%interuptions_removed
id,,,,,,,,,,
20150512_110526,copper-10worms-N2-day2-standard,180.0,10,860,47,195,33,850,813,96
20150514_121513,copper-10worms-N2-day4-standard_C,180.0,10,1041,55,265,41,1031,986,96
20150514_164204,copper-10worms-N2-day4-standard_C,180.0,10,1454,41,214,35,1444,1413,98
20150515_124728,copper-10worms-N2-day5-standard_B,180.0,10,1083,23,164,16,1073,1060,99
20150515_124736,copper-10worms-N2-day5-standard_C,180.0,10,1571,79,274,58,1561,1492,96
20150515_124840,copper-10worms-N2-day5-standard_A_Correct,180.0,10,1181,32,251,25,1171,1149,98
20150515_155539,copper-10worms-N2-day5-standard_A,180.0,10,654,18,120,14,644,636,99
20150515_155545,copper-10worms-N2-day5-standard_C,180.0,10,618,20,140,15,608,598,98
20150518_121756,copper-10worms-N2-day1-standard_15degree,180.0,10,1014,52,121,44,1004,962,96


In [10]:
columns = [np.array([u'basics', u'basics', u'basics',
                     u'total tracks', u'total tracks', 
                     u'moving tracks', u'moving tracks',
                     u'interuptions', u'interuptions', u'interuptions']),
           
            np.array([u'name', u'total-minutes', u'#worms', 
                      u'MWT', u'WALDO', 
                      u'MWT', u'WALDO',
                     u'MWT', u'number removed', u'percent removed'])]
p = pd.DataFrame(np.array(pf), columns=columns, index=pf.index)
p.to_html('table_of_waldo_performance.html')
p.to_latex('table_of_waldo_performance.tex')
p

basics                \
                                                      name total-minutes   
id                                                                         
20150512_110526            copper-10worms-N2-day2-standard           180   
20150514_121513          copper-10worms-N2-day4-standard_C           180   
20150514_164204          copper-10worms-N2-day4-standard_C           180   
20150515_124728          copper-10worms-N2-day5-standard_B           180   
20150515_124736          copper-10worms-N2-day5-standard_C           180   
20150515_124840  copper-10worms-N2-day5-standard_A_Correct           180   
20150515_155539          copper-10worms-N2-day5-standard_A           180   
20150515_155545          copper-10worms-N2-day5-standard_C           180   
20150518_121756   copper-10worms-N2-day1-standard_15degree           180   
20150518_121757   copper-10worms-N2-day1-standard_15degree           180   
20150518_121809   copper-10worms-N2-day1-standard_15degree           180   
20150518_182452   copper-10worms-N2-day1-standard_25degree           180   
20150518_182454   copper-10worms-N2-day1-standard_25degree           180   
20150518_182501   copper-10worms-N2-day1-standard_25degree           180   
20150519_122411              copper-10worms-N2-day2-nofood           180   
20150519_122420            copper-10worms-N2-day2-standard           180   
20150519_170520            copper-10worms-N2-day2-standard           180   
20150519_170525              copper-10worms-N2-day2-nofood           180   
20150519_170530              copper-10worms-N2-day2-nofood           180   
20150602_112810              fullday-9worms-N2-day2-groupC          39.2   
20150608_121717            copper-30worms-N2-day1-standard           180   
20150608_121725            copper-10worms-N2-day1-standard           180   
20150608_121812            copper-50worms-N2-day1-standard           180   
20150608_165515            copper-40worms-N2-day1-standard           180   
20150608_165523            copper-60worms-N2-day1-standard           180   
20150608_165610            copper-20worms-N2-day1-standard           180   
20150615_124720           copper-10worms-N2-day1-25degrees           180   
20150615_124728           copper-10worms-N2-day1-25degrees           180   
20150615_124738           copper-10worms-N2-day1-25degrees           180   
20150615_170747           copper-10worms-N2-day1-15degrees           180   
20150615_170754           copper-10worms-N2-day1-15degrees           180   
20150615_170759           copper-10worms-N2-day1-15degrees           180   

                       total tracks       moving tracks       interuptions  \
                #worms          MWT WALDO           MWT WALDO          MWT   
id                                                                           
20150512_110526     10          860    47           195    33          850   
20150514_121513     10         1041    55           265    41         1031   
20150514_164204     10         1454    41           214    35         1444   
20150515_124728     10         1083    23           164    16         1073   
20150515_124736     10         1571    79           274    58         1561   
20150515_124840     10         1181    32           251    25         1171   
20150515_155539     10          654    18           120    14          644   
20150515_155545     10          618    20           140    15          608   
20150518_121756     10         1014    52           121    44         1004   
20150518_121757     10          853    32           119    23          843   
20150518_121809     10          905    31            84    22          895   
20150518_182452     10          473    22           112    18          463   
20150518_182454     10         1408    33            86    28         1398   
20150518_182501     10         1294    25            95    25         1284   
20150519_122411     10         1761   204           804   160    

# Specific Paper Questions

In [80]:
# Abstract
# waldo is able to remove X% of disruptions
pf['%interuptions_removed'].describe()
# mean -> 96%

count    39.000000
mean     96.102564
std       2.693021
min      89.000000
25%      95.500000
50%      97.000000
75%      98.000000
max      99.000000
Name: %interuptions_removed, dtype: float64

In [86]:
# Introduction
# For an experimental setup of following 10 worms for 3 hours.
# MWT identifies over 3500 tracks, on average,
# however, only 16% of these tracks move a distance greater than 1 bl.

pf['MWT_total_tracks'].describe()

count       39.000000
mean      3561.435897
std       7112.950231
min        473.000000
25%        881.000000
50%       1181.000000
75%       1733.000000
max      33940.000000
Name: MWT_total_tracks, dtype: float64

In [87]:
pf[['MWT_total_tracks', 'MWT_moving_tracks']]
(pf['MWT_moving_tracks'] / pf['MWT_total_tracks'] * 100).describe()

count    39.000000
mean     16.271331
std       9.191527
min       2.968617
25%       9.525277
50%      15.327793
75%      20.509613
max      45.655877
dtype: float64

In [91]:
# Introduction
# we analyzed 39 three-hour recordings and, on average, waldo corrected 3300 disruptions per recording.

In [ ]:
# TODO
# BAD METRIC

# ... and retained the identity of 80% of tracks from the time they entered the field of view to the time the left.

# BAD METRIC

# These imporvement shave lead to a 10 fold increase in the numbers of tracks that follow worms at least 20 minutes.

In [93]:
df = pf[pf['total-minutes'] == 180]
print(len(df))
df['#interuptions_removed'].describe()

39


count       39.000000
mean      3328.948718
std       6519.388500
min        452.000000
25%        847.500000
50%       1149.000000
75%       1600.000000
max      31302.000000
Name: #interuptions_removed, dtype: float64

In [96]:
# Results... shape identification
# Together, consolidation and puring are responsible for 43% of track reductions implemented by WALDO
((op['prune'] + op['consolidate']) * 100 / op['total-removed']).describe()

count    39.000000
mean     43.670878
std      13.394784
min       9.419263
25%      35.863574
50%      44.338725
75%      53.732062
max      69.941776
dtype: float64

In [10]:
gaps = pd.read_csv('/home/projects/worm_movement/Data/validation_screens/gap_validate.csv')
gaps

# Notes all potential gaps are scored. not just the ones that were completed during waldo

collisions = pd.read_csv('/home/projects/worm_movement/Data/validation_screens/collision_validate.csv')
collisions

# Note all V shaped portions of the network are scored...

,eid,bid,ans
0,20130702_135652,17571,10
1,20130702_135652,11746,20
2,20130702_135652,9663,20
3,20130702_135652,10021,20
4,20130702_135652,2378,10
5,20130702_135652,17675,20
6,20130702_135704,8987,20
7,20130702_135704,60028,20
8,20130702_135704,47456,20
9,20130702_135704,16461,15


In [ ]:
# GAPS
# TODO

# FROM NICKS WORK
# In test sets, this threshold was determined to find 95% of the missing arcs between tracks created by the same animal
# while only introducint 5% of false positives.

In [ ]:
# Multiple Operations
# TODO
# only x% of animals have all tracks for the same animal merged into one node that is isolated from rest of network.

# using MWT one is able to continuosly follow 1 out of each 100 worms that pass through the ROI
# WADLDO can successfully do this for 1 of 8.

# Check Later!


-- during infer-gaps step:

are nodes merged (current calculation) or are nodes joined usind directed edge


-- during resolve-collision step:

are nodes merged (current calculation. 5->2. 3 node difference) or
is only the node in the middle removed (paper figure discription 5->4. 1 node difference).

-- double check:

use chore to count total nodes and to count nodes that have moved in both MWT and WALDO


In [77]:
d.sort()

Help on built-in function sorted in module __builtin__:

sorted(...)
    sorted(iterable, cmp=None, key=None, reverse=False) --> new sorted list



In [113]:
eids = list(report_cards.keys())
dats = []
for eid in eids:
    terms = pd.read_csv('/home/projects/worm_movement/Data/WALDO_Data/{eid}/waldo/{eid}-terminals.csv'.format(eid=eid))
    node_sum = pd.read_csv('/home/projects/worm_movement/Data/WALDO_Data/{eid}/waldo/{eid}-node-summary.csv'.format(eid=eid))
    t0 = 0
    tN = max(node_sum['tN'])
    
    before = pd.DataFrame(index=terms.index)
    before['dt'] = terms['tN'] - terms['t0']
    before['%'] = before / tN
    
    after = pd.DataFrame(index=node_sum.index)
    after['dt'] = node_sum['tN'] - node_sum['t0']
    after['%'] = after / tN
    
    row = organization_table.loc[eid]
    N = row['#worms']
    total_min = row['total-minutes']
    
    a = {'b50%': len(before[before['%'] > 0.5]),
         'a50%': len(after[after['%'] > 0.5]),
         'b90%': len(before[before['%'] > 0.9]),
         'a90%': len(after[after['%'] > 0.9]),
         'b99%': len(before[before['%'] > 0.99]),
         'a99%': len(after[after['%'] > 0.99]),
         'b% mean': np.mean(before['%']),
         'a% mean': np.mean(after['%']),
         'b_N_longest': list(before.sort('dt', ascending=False).loc[:N, 'dt']),
         'a_N_longest': list(after.sort('dt', ascending=False).loc[:N, 'dt']),
         'eid': eid,
         'N': N,
         'total-min': total_min,
         }
    dats.append(a)
# after.hist()
# plt.show()
# before.hist()
#print(before[before['%'] > 0.5])
#after[after['%'] > 0.5]
d = pd.DataFrame(dats).set_index('eid')[['N', 'b50%', 'a50%', 'b90%', 'a90%', 'b99%', 'a99%', 'total-min']]
d

,N,b50%,a50%,b90%,a90%,b99%,a99%,total-min
eid,,,,,,,,
20150615_170754,10,2,10,0,7,0,4,180.0
20150608_165523,60,1,29,0,0,0,0,180.0
20150602_112810,9,8,8,8,8,6,8,39.2
20150518_121809,10,2,9,0,4,0,4,180.0
20150615_170759,10,0,10,0,8,0,8,180.0
20150615_124720,10,4,10,0,4,0,2,180.0
20150615_124728,10,6,9,0,7,0,5,180.0
20150515_124736,10,0,3,0,1,0,1,180.0
20150519_170520,10,1,8,0,4,0,3,180.0


In [112]:
d2 = d[d['N'] == 10]
total_n = sum(d2['N'])
total_complete = sum(d2['a50%'])
before_complete= sum(d2['b50%'])

print(total_n, 'worms in recordings of 10 worms')
print(total_complete, 'tracks spanning > 90% of recording')
print(100. * total_complete/ total_n, '% worms completely solved')

print(100. * before_complete/ total_n, '% worms completely solved before waldo')

print(before_complete)
1/4
#1 / 9

260 worms in recordings of 10 worms
190 tracks spanning > 90% of recording
73.0769230769 % worms completely solved
13.0769230769 % worms completely solved before waldo
34


0.25

In [109]:
d2 = d[d['N'] == 10]
total_n = sum(d2['N'])
total_complete = sum(d2['a90%'])
before_complete= sum(d2['b90%'])

print(total_n, 'worms in recordings of 10 worms')
print(total_complete, 'tracks spanning > 90% of recording')
print(100. * total_complete/ total_n, '% worms completely solved')

print(100. * before_complete/ total_n, '% worms completely solved before waldo')

260 worms in recordings of 10 worms
107 tracks spanning > 90% of recording
41.1538461538 % worms completely solved
0.0 % worms completely solved before waldo
inf


In [96]:
total_complete = sum(d2['a99%'])
before_complete= sum(d2['b99%'])

print(total_n, 'worms in recordings of 10 worms')
print(total_complete, 'tracks spanning > 90% of recording')
print(100. * total_complete/ total_n, '% worms completely solved')

print(100. * before_complete/ total_n, '% worms completely solved before waldo')

260 worms in recordings of 10 worms
69 tracks spanning > 90% of recording
26.5384615385 % worms completely solved
0.0 % worms completely solved before waldo


In [119]:
d = pd.DataFrame(dats).set_index('eid')[['N', 'a% mean', 'b% mean']]
a = np.mean(d['a% mean'])
b = np.mean(d['b% mean'])
print(a)
print(b)
d

0.292296497855
0.0310248982981


,N,a% mean,b% mean
eid,,,
20150615_170754,10,0.453969,0.008826
20150608_165523,60,0.070093,0.002400
20150602_112810,9,0.999392,0.726655
20150518_121809,10,0.320505,0.010869
20150615_170759,10,0.587533,0.014929
20150615_124720,10,0.399249,0.014877
20150615_124728,10,0.470694,0.009773
20150515_124736,10,0.123110,0.006647
20150519_170520,10,0.423453,0.010572


In [104]:
d = pd.DataFrame(dats).set_index('eid')[['N', 'total-min', 'a_N_longest', 'b_N_longest']]
before_mean = []
after_mean = []

for eid, row in d.iterrows():
    N = row['N']
    total_min = row['total-min']
    if total_min != 180:
        continue
    #     if N != 10:
    #         continue
    a = row['a_N_longest']
    b = row['b_N_longest']
    print(N, total_min, np.mean(a) / 60.0, np.mean(b) / 60.0)
    after_mean.append(np.mean(a))
    before_mean.append(np.mean(b))
    
print(np.mean(after_mean) / 60.0)
print(np.mean(before_mean) / 60.0)

10 180.0 99.8035231481 3.71638392732
60 180.0 33.2783758715 2.14938120796
10 180.0 88.87939 2.92656882255
10 180.0 179.87491 8.71662807882
10 180.0 74.8579805556 10.2693546243
10 180.0 93.6403684211 7.47577424242
10 180.0 50.2107134409 2.1902635514
10 180.0 170.760908333 9.48993814815
10 180.0 179.585709524 9.21139649123
30 180.0 15.4346068922 3.19206303437
10 180.0 55.3826057292 1.87665978065
10 180.0 19.8748874429 1.25525459021
10 180.0 115.034664444 38.3020944444
10 180.0 44.8625066667 2.96752672798
10 180.0 28.9462283333 12.0069428571
40 180.0 58.0312033033 7.51437390017
10 180.0 48.6555490991 4.27820096386
50 180.0 44.5776549645 5.06980285388
10 180.0 105.282682353 6.79928886719
20 180.0 87.8073755556 6.06607835991
10 180.0 107.961455556 26.7758822917
10 180.0 35.1377909722 6.05974557576
10 180.0 179.732594444 3.9499888764
10 180.0 91.4896692982 4.41715273504
10 180.0 105.845802222 2.62237659989
10 180.0 34.5316813131 1.75366509288
10 180.0 38.2166067376 2.84180738363
10 180.0 89.